In [2]:
def forme_standardVERScanonique(A,B,z): 
    '''En notant n le nombre d'inconnues, m longueur de B (nombre de contraintes) alors A est de la taille m*n, z de longueur n'''
    '''On passe de résoudre A <= B selon z à A' = B selon z\''''

    new_A=[]
    m=len(B)
    for i in range(len(A)): 
        new_A.append(A[i]+[1 if k==i else 0 for k in range(m)])
    new_z=z+[0]*(m+1)

    base_sortante=[i+len(A[0]) for i in range(m)]

    return new_A,B,new_z,base_sortante

def tableau_de_calcul(A,B,z):
    M=[]
    for i in range(len(A)):
        M.append(A[i]+[B[i]])
    M.append(z)
    return M

def choix_variable1(M): ### PRENDRRE LE MAX NE FOURNIT PAS TOUJOURS LE BON PIVOT !!!
    c=M[-1].index(max(M[-1]))
    return c

def choix_variable2(M,c): # c=choix_variable1(M)
    f=lambda i : M[i][-1]/M[i][c] if M[i][c]!=0 else float('inf')
    indices_des_lignes=[i for i in range(len(M)-1) if f(i)>=0]
    if indices_des_lignes:
        l = min(indices_des_lignes, key=f)
        return l
    else:
        return None

def update(M,c,l,base_sortante): # c=choix_variable1(M) ; l=choix_variable2(M,c)
    new_M=[[] for i in range(len(M))]
    for j in range(len(M[0])):
        new_M[l].append(M[l][j]/M[l][c])

    f=lambda i,j : M[i][j] - (M[i][c]/M[l][c])*M[l][j]
    for i in range(len(M)):
        if i!=l:
            new_M[i]=[f(i,j) for j in range(len(M[i]))]
    base_sortante=[c if i==l else base_sortante[i] for i in range(len(base_sortante))] #au lieu de base_sortante[l]=c qui créait des effets de bords donc une boucle dans le code corps

    return new_M,base_sortante

def elmt_strictement_positif_dans(l):
    for i in l:
        if i>0:
            return True
    else :
        return False

def simplexe(A, B, z):
    variables_hors_base = [i for i in range(len(A[0]))]

    (new_A, B, new_z, base_sortante) = forme_standardVERScanonique(A, B, z)
    M = tableau_de_calcul(new_A, B, new_z)

    # On utilise vecteurs_existants pour stocker les vecteurs solutions et z_maximisé
    vecteurs_existants = []

    while elmt_strictement_positif_dans(M[-1][:-1]):
        c = choix_variable1(M)
        l = choix_variable2(M, c)
        if l is None:  # Aucun pivot possible
            break
        M, base_sortante = update(M, c, l, base_sortante)
        
        z_maximisé = -M[-1][-1]

        # Construire le vecteur solution courant
        vecteur_sol = []
        for i in variables_hors_base:
            if i in base_sortante:
                ligne = base_sortante.index(i)
                vecteur_sol.append(M[ligne][-1])
            else:
                vecteur_sol.append(0)

        # Vérifier si une solution identique (même vecteur solution) existe
        if any(v[0] == vecteur_sol for v in vecteurs_existants):
            meilleure_solution = max(vecteurs_existants, key=lambda v: v[1])  # Meilleure valeur de z
            if z_maximisé <= meilleure_solution[1]:
                break  # La meilleure solution a été trouvée
        else:
            vecteurs_existants.append((vecteur_sol, z_maximisé))


        # Ajouter le vecteur solution et z_maximisé à la liste des solutions existantes
        vecteurs_existants.append((vecteur_sol, z_maximisé))

    # Affichage des résultats
    print(variables_hors_base)
    for vecteur, z_max in [vecteurs_existants[-1]]:
        print(f"{vecteur} - {z_max}")



In [3]:
### Exemple 1
A=[
    [3,2],
    [1,0],
    [0,1]
]
B=[1800,400,600]
z=[30,50]

simplexe(A,B,z)

[0, 1]
[200.0, 600.0] - 36000.0


In [4]:
### Exemple 2
A = [
    [2, 3, 4],
    [1, 2, 1],
    [0, 1, 2],
]
B = [1200, 800, 600]
z = [40, 30, 50]

simplexe(A,B,z)

[0, 1, 2]
[0, 0, 300.0] - 15000.0


In [5]:
### Exemple 3
A = [
    [1,1,1],
    [1,0,0],
    [8,7,5],
]
B = [12,5,145]
z = [120,108,75]

simplexe(A,B,z)

[0, 1, 2]
[5.0, 7.0, 0] - 1356.0


In [6]:
import warnings
from scipy.optimize import linprog
warnings.filterwarnings("ignore", category=DeprecationWarning)

A = [
    [2, 3, 4],
    [1, 2, 1],
    [0, 1, 2],
]
B = [1200, 800, 600]
z = [40, 30, 50]


# Coefficients de la fonction objectif (à minimiser, donc -z)
c = [-i for i in z]# Les signes sont inversés car linprog minimise par défaut

# Bornes pour chaque variable (x, y, z >= 0)
bounds = [(0, None), (0, None), (0, None)]

# Résolution du problème
result = linprog(c, A_ub=A, b_ub=B, bounds=bounds, method="simplex")

# Résultats formatés
if result.success:
    variables = ["x", "y", "z"]
    solution = {var: round(val, 2) for var, val in zip(variables, result.x)}
    z_max = round(-result.fun, 2)  # Revenir au signe +

    print(variables)
    print(f"{list(solution.values())} - {z_max}")
else:
    print("Problème sans solution ou échec de l'algorithme.")

['x', 'y', 'z']
[600.0, 0.0, 0.0] - 24000.0
